 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Twoim zadaniem tutaj jest pobranie udostępnionych danych do obszaru roboczego, które w następnym kroku wgrasz na bazę danych. Ich obróbkę oraz analizę przeprowadzisz w specjalnie do tego celu przygotowanych kolejnych notatnikach.
 
 To tutaj wcielasz się w rolę Data Engineera, którego zadaniem jest poprawne dostarczenie danych Analitykowi Danych. W tym notatniku zrealizujesz pierwsze wymaganie, które otrzymaliśmy od klienta: stworzenie mechanizmu pobierającego dane z udostępnionego API. 

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/docs/v2).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją możemy stwiedzić, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest parametr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [2]:
import pandas as pd
import json as loads
import time
import datetime
import requests
import csv

from dateutil.relativedelta import relativedelta
from requests import Session
from datetime import datetime, timedelta
from time import sleep


 Tutaj zdefiniuj paramatry połączenia do API

In [3]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

api_url = 'https://api-datalab.coderslab.com/api/aircraft'
headers = {
    'Authorization': f'{APIKEY}'
}

responce = requests.get(api_url, headers = headers)

if responce.status_code == 200:
    data = responce.json()
    print({responce.status_code})
else:
    print(f'blad {responce.status_code} - {responce.text}')

{200}


In [3]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'
response = requests.get('https://api-datalab.coderslab.com/docs/v2/Airport/get_api_airport__airportId_', {'apikey': APIKEY})
print(response.status_code)


200


In [4]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'
response = requests.get('https://api-datalab.coderslab.com/docs/v2/Airport/get_api_airport_', {'apikey': APIKEY})
print(response.status_code)

200


In [4]:
df = pd.read_csv(
  r'.\airports.csv'
)
df.head()

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638


 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [6]:
airport = pd.read_csv(
  r'.\airports.csv')
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

headers = {
    'Authorization': f'{APIKEY}'
}


airport_data = []
for airport_id in airport['origin_airport_id']:
    url = 'https://api-datalab.coderslab.com/api/airport/{}'.format(airport_id)
    response = requests.get(url, headers = headers)
    
    if response.status_code == 200:
        airport_data.append(response.json())
    else:
        print(f'failed with status code {response.status_code}: {response.text}')
        
        for row in airport_data:
            if row in airport_data != 'error':
                print(row)
                
airport_df = pd.DataFrame.from_records(airport_data)
print(airport_df)

failed with status code 400: {"error":"Airport with id 10874 not exist"}
failed with status code 400: {"error":"Airport with id 11233 not exist"}
failed with status code 400: {"error":"Airport with id 13360 not exist"}
failed with status code 400: {"error":"Airport with id 15008 not exist"}
failed with status code 400: {"error":"Airport with id 14150 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
failed with status code 400: {"error":"Airport with id 15323 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
failed with status code 400: {"error":"Airport with id 14814 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
failed with sta

failed with status code 400: {"error":"Airport with id 13829 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 11525 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 12898 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 10245 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

...

failed with status code 400: {"error":"Airport with id 11898 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 10620 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 11097 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 10135 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

failed with status code 400: {"error":"Airport with id 12945 not exist"}
{'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}
{'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}
{'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}
{'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}
{'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}
{'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [7]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [17]:
airport_df.to_csv('airport_list222.csv', index = None, header=True)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [9]:
api_url_weather = 'https://api-datalab.coderslab.com/api/v2/airportWeather'
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'
start_date = pd.to_datetime('2019-01-01')
end_date = pd.to_datetime('2020-03-31')

weather_data = []

date = start_date
while date <= end_date:
    date_str = date.strftime('%y-%m')
    response = requests.get(api_url_weather, headers = {'authorization': APIKEY}, params = {'date': date_str})
    data = response.json()
    weather_data.extend(data)
    print(f'pobrano dane za miesiac: {date_str}')
    date += relativedelta(months = 1)
    
#     time.sleep(0.05)
    
weather_df = pd.json_normalize(weather_data)
print(weather_df)
    

pobrano dane za miesiac: 19-01
pobrano dane za miesiac: 19-02
pobrano dane za miesiac: 19-03
pobrano dane za miesiac: 19-04
pobrano dane za miesiac: 19-05
pobrano dane za miesiac: 19-06
pobrano dane za miesiac: 19-07
pobrano dane za miesiac: 19-08
pobrano dane za miesiac: 19-09
pobrano dane za miesiac: 19-10
pobrano dane za miesiac: 19-11
pobrano dane za miesiac: 19-12
pobrano dane za miesiac: 20-01
pobrano dane za miesiac: 20-02
pobrano dane za miesiac: 20-03
       WT18      STATION                                               NAME  \
0      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
1      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
2      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
3      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
4      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
...     ...          ...                                     

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [10]:
weather_df.to_csv(r'C:\Users\maria\kurs calosc\project\data\raw\airport_weather.csv', index = None, header=True)

In [33]:
weather_df.head()

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [48]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

api_url2 = 'https://api-datalab.coderslab.com/api/aircraft'
headers = {
    'Authorization': f'{APIKEY}'
}

responce = requests.get(api_url2, headers = headers)

if responce.status_code == 200:
    data = responce.json()
    print({responce.status_code})
else:
    print(f'blad {responce.status_code} - {responce.text}')

{200}


In [75]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

api_url2 = 'https://api-datalab.coderslab.com/api/aircraft'
headers = {
    'Authorization': f'{APIKEY}'
}

responce = requests.get(api_url2, headers = headers)
aircraft_df = responce.json()

print(aircraft_df)

aircraft_df.to_csv ('aircraft.csv', index = False, header=True)

aircraft_df.head()

[{'MANUFACTURE_YEAR': 1944, 'TAIL_NUM': 'N54514', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1945, 'TAIL_NUM': 'N1651M', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1953, 'TAIL_NUM': 'N100CE', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1953, 'TAIL_NUM': 'N141FL', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1953, 'TAIL_NUM': 'N151FL', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1953, 'TAIL_NUM': 'N381FL', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1954, 'TAIL_NUM': 'N6174C', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1954, 'TAIL_NUM': 'N400UA', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1955, 'TAIL_NUM': 'N747CE', 'NUMBER_OF_SEATS': 2}, {'MANUFACTURE_YEAR': 1955, 'TAIL_NUM': 'N9148F', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1955, 'TAIL_NUM': 'N131FL', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1955, 'TAIL_NUM': 'N351FL', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1955, 'TAIL_NUM': 'N391FL', 'NUMBER_OF_SEATS': 0}, {'MANUFACTURE_YEAR': 1956, 'TAIL_NUM': 'N171FL', 'NUMBER_OF_SEA

AttributeError: 'list' object has no attribute 'to_csv'

In [10]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

api_url2 = 'https://api-datalab.coderslab.com/api/aircraft'
headers = {
    'Authorization': f'{APIKEY}'
}

aircraft_data = []

response = requests.get(api_url2, headers = headers)
aircraft_data = response.json()


aircraft_df = pd.DataFrame(aircraft_data)
aircraft_df.to_csv('file3.csv', index=None, header=True)





In [11]:
aircraft_df.head()

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [ ]:
aircraft_df = pd.DataFrame(aircraft_data)
aircraft_df.to_csv('file3.csv', index=None, header=True)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

In [3]:
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

api_url3 = 'https://api-datalab.coderslab.com/api/v2/flight'
headers = {
    'Authorization': f'{APIKEY}'
}

responce = requests.get(api_url3, headers = headers)

if responce.status_code == 200:
    data = responce.json()
    print({responce.status_code})
else:
    print(f'blad {responce.status_code} - {responce.text}')

blad 400 - {"error":"The airportId param is required"}


In [ ]:
airport_df = pd.read_csv(
  r'.\airports.csv')
APIKEY = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'
api_url3 = 'https://api-datalab.coderslab.com/api/v2/flight'
headers = {
    'Authorization': f'{APIKEY}'
}
airports_id = airport_df['origin_airport_id'].tolist()


start_date = pd.to_datetime('2019-01-01')
end_date = pd.to_datetime('2020-03-31')

for airport_id in airports_id:
    date = start_date
    flights_data = []
    while date <= end_date:
        response = requests.get(api_url3, headers = {'authorization': APIKEY}, params = {'date': date, 'airportId': airport_id})
        if response.status_code == 200:
            data = response.json()
            flights_data.extend(data)
#             print(f'data downloaded for {airport_id} on {date}')
        elif response.status_code == 204:
            print(f'no data for airport {airport_id} on {date}')
            break
        else:
            print(f'{response.status_code}: {response.tex}')
        date += timedelta(days = 1)
    sleep(0.05)
    
flight_df = pd.json_normalize(flight_Data)
print(flight_df)




no data for airport 10874 on 2019-01-01 00:00:00
no data for airport 11233 on 2019-01-01 00:00:00
no data for airport 13360 on 2019-01-01 00:00:00
no data for airport 15008 on 2019-01-01 00:00:00
no data for airport 11638 on 2019-01-01 00:00:00
no data for airport 14150 on 2019-01-01 00:00:00
no data for airport 15323 on 2019-01-01 00:00:00
no data for airport 14814 on 2019-01-01 00:00:00
no data for airport 12007 on 2019-01-01 00:00:00
no data for airport 11337 on 2019-01-01 00:00:00
no data for airport 13342 on 2019-01-01 00:00:00
no data for airport 15070 on 2019-01-01 00:00:00
no data for airport 13244 on 2019-01-01 00:00:00
no data for airport 12280 on 2019-01-01 00:00:00
no data for airport 15096 on 2019-01-01 00:00:00
no data for airport 11641 on 2019-01-01 00:00:00


In [4]:
api_url_flight = 'https://api-datalab.coderslab.com/api/v2/flight'
api_key = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'

airports_df = pd.read_csv(
  r'.\airports.csv')

airports_ids = (row['origin_airport_id'] for _, row in airports_df.iterrows())

all_results = []

start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)

session = Session()

for airport_id in airports_ids:
    url_params = {
        "airportId": airport_id,
        "date": start_date.strftime('%Y-%m-%d')
    }
    while url_params["date"] < end_date.strftime('%Y-%m-%d'):
        try:
            response = session.get(api_url_flight, headers={"Authorization": api_key}, params=url_params)
            if response.status_code == 200:
                results = json.loads(response.content)
                if results:
                    all_results.extend(results)
                else:
                    print(f'brak danych dla lotniska {airport_id} dnia {url_params["date"]}')
            elif response.status_code == 204:
                print(f'brak danych dla lotniska {airport_id} dnia {url_params["date"]}')
            else:
                print(f'Error: {response.status_code}  {response.content}')
            url_params['date'] = (datetime.strptime(url_params['date'], '%Y-%m-%d') + relativedelta(months=1)).strftime('%Y-%m-%d')
        except requests.exceptions.ConnectionError:
            print("Blad polaczenia")
            time.sleep(10)
        time.sleep(0.06)
    print(f'dane dla lotniska {airport_id} pobrane')
flight_df = pd.json_normalize(all_results)
flight_df

flight_df.to_csv('flight.csv', index=False)

brak danych dla lotniska 10874 dnia 2019-01-01
brak danych dla lotniska 10874 dnia 2019-02-01
brak danych dla lotniska 10874 dnia 2019-03-01
brak danych dla lotniska 10874 dnia 2019-04-01
brak danych dla lotniska 10874 dnia 2019-05-01
brak danych dla lotniska 10874 dnia 2019-06-01
brak danych dla lotniska 10874 dnia 2019-07-01
brak danych dla lotniska 10874 dnia 2019-08-01
brak danych dla lotniska 10874 dnia 2019-09-01
brak danych dla lotniska 10874 dnia 2019-10-01
brak danych dla lotniska 10874 dnia 2019-11-01
brak danych dla lotniska 10874 dnia 2019-12-01
brak danych dla lotniska 10874 dnia 2020-01-01
brak danych dla lotniska 10874 dnia 2020-02-01
brak danych dla lotniska 10874 dnia 2020-03-01
dane dla lotniska 10874 pobrane
brak danych dla lotniska 11233 dnia 2019-01-01
brak danych dla lotniska 11233 dnia 2019-02-01
brak danych dla lotniska 11233 dnia 2019-03-01
brak danych dla lotniska 11233 dnia 2019-04-01
brak danych dla lotniska 11233 dnia 2019-05-01
brak danych dla lotniska 112

brak danych dla lotniska 15070 dnia 2019-03-01
brak danych dla lotniska 15070 dnia 2019-04-01
brak danych dla lotniska 15070 dnia 2019-05-01
brak danych dla lotniska 15070 dnia 2019-06-01
brak danych dla lotniska 15070 dnia 2019-07-01
brak danych dla lotniska 15070 dnia 2019-08-01
brak danych dla lotniska 15070 dnia 2019-09-01
brak danych dla lotniska 15070 dnia 2019-10-01
brak danych dla lotniska 15070 dnia 2019-11-01
brak danych dla lotniska 15070 dnia 2019-12-01
brak danych dla lotniska 15070 dnia 2020-01-01
brak danych dla lotniska 15070 dnia 2020-02-01
brak danych dla lotniska 15070 dnia 2020-03-01
dane dla lotniska 15070 pobrane
brak danych dla lotniska 13244 dnia 2019-01-01
brak danych dla lotniska 13244 dnia 2019-02-01
brak danych dla lotniska 13244 dnia 2019-03-01
brak danych dla lotniska 13244 dnia 2019-04-01
brak danych dla lotniska 13244 dnia 2019-05-01
brak danych dla lotniska 13244 dnia 2019-06-01
brak danych dla lotniska 13244 dnia 2019-07-01
brak danych dla lotniska 132

brak danych dla lotniska 11092 dnia 2019-05-01
brak danych dla lotniska 11092 dnia 2019-06-01
brak danych dla lotniska 11092 dnia 2019-07-01
brak danych dla lotniska 11092 dnia 2019-08-01
brak danych dla lotniska 11092 dnia 2019-09-01
brak danych dla lotniska 11092 dnia 2019-10-01
brak danych dla lotniska 11092 dnia 2019-11-01
brak danych dla lotniska 11092 dnia 2019-12-01
brak danych dla lotniska 11092 dnia 2020-01-01
brak danych dla lotniska 11092 dnia 2020-02-01
brak danych dla lotniska 11092 dnia 2020-03-01
dane dla lotniska 11092 pobrane
brak danych dla lotniska 10581 dnia 2019-01-01
brak danych dla lotniska 10581 dnia 2019-02-01
brak danych dla lotniska 10581 dnia 2019-03-01
brak danych dla lotniska 10581 dnia 2019-04-01
brak danych dla lotniska 10581 dnia 2019-05-01
brak danych dla lotniska 10581 dnia 2019-06-01
brak danych dla lotniska 10581 dnia 2019-07-01
brak danych dla lotniska 10581 dnia 2019-08-01
brak danych dla lotniska 10581 dnia 2019-09-01
brak danych dla lotniska 105

brak danych dla lotniska 10792 dnia 2019-07-01
brak danych dla lotniska 10792 dnia 2019-08-01
brak danych dla lotniska 10792 dnia 2019-09-01
brak danych dla lotniska 10792 dnia 2019-10-01
brak danych dla lotniska 10792 dnia 2019-11-01
brak danych dla lotniska 10792 dnia 2019-12-01
brak danych dla lotniska 10792 dnia 2020-01-01
brak danych dla lotniska 10792 dnia 2020-02-01
brak danych dla lotniska 10792 dnia 2020-03-01
dane dla lotniska 10792 pobrane
brak danych dla lotniska 14259 dnia 2019-01-01
brak danych dla lotniska 14259 dnia 2019-02-01
brak danych dla lotniska 14259 dnia 2019-03-01
brak danych dla lotniska 14259 dnia 2019-04-01
brak danych dla lotniska 14259 dnia 2019-05-01
brak danych dla lotniska 14259 dnia 2019-06-01
brak danych dla lotniska 14259 dnia 2019-07-01
brak danych dla lotniska 14259 dnia 2019-08-01
brak danych dla lotniska 14259 dnia 2019-09-01
brak danych dla lotniska 14259 dnia 2019-10-01
brak danych dla lotniska 14259 dnia 2019-11-01
brak danych dla lotniska 142

brak danych dla lotniska 14108 dnia 2019-09-01
brak danych dla lotniska 14108 dnia 2019-10-01
brak danych dla lotniska 14108 dnia 2019-11-01
brak danych dla lotniska 14108 dnia 2019-12-01
brak danych dla lotniska 14108 dnia 2020-01-01
brak danych dla lotniska 14108 dnia 2020-02-01
brak danych dla lotniska 14108 dnia 2020-03-01
dane dla lotniska 14108 pobrane
brak danych dla lotniska 13931 dnia 2019-01-01
brak danych dla lotniska 13931 dnia 2019-02-01
brak danych dla lotniska 13931 dnia 2019-03-01
brak danych dla lotniska 13931 dnia 2019-04-01
brak danych dla lotniska 13931 dnia 2019-05-01
brak danych dla lotniska 13931 dnia 2019-06-01
brak danych dla lotniska 13931 dnia 2019-07-01
brak danych dla lotniska 13931 dnia 2019-08-01
brak danych dla lotniska 13931 dnia 2019-09-01
brak danych dla lotniska 13931 dnia 2019-10-01
brak danych dla lotniska 13931 dnia 2019-11-01
brak danych dla lotniska 13931 dnia 2019-12-01
brak danych dla lotniska 13931 dnia 2020-01-01
brak danych dla lotniska 139

...

brak danych dla lotniska 14098 dnia 2019-04-01
brak danych dla lotniska 14098 dnia 2019-05-01
brak danych dla lotniska 14098 dnia 2019-06-01
brak danych dla lotniska 14098 dnia 2019-07-01
brak danych dla lotniska 14098 dnia 2019-08-01
brak danych dla lotniska 14098 dnia 2019-09-01
brak danych dla lotniska 14098 dnia 2019-10-01
brak danych dla lotniska 14098 dnia 2019-11-01
brak danych dla lotniska 14098 dnia 2019-12-01
brak danych dla lotniska 14098 dnia 2020-01-01
brak danych dla lotniska 14098 dnia 2020-02-01
brak danych dla lotniska 14098 dnia 2020-03-01
dane dla lotniska 14098 pobrane
dane dla lotniska 12982 pobrane
brak danych dla lotniska 14512 dnia 2019-01-01
brak danych dla lotniska 14512 dnia 2019-02-01
brak danych dla lotniska 14512 dnia 2019-03-01
brak danych dla lotniska 14512 dnia 2019-04-01
brak danych dla lotniska 14512 dnia 2019-05-01
brak danych dla lotniska 14512 dnia 2019-06-01
brak danych dla lotniska 14512 dnia 2019-07-01
brak danych dla lotniska 14512 dnia 2019-08

brak danych dla lotniska 11778 dnia 2019-04-01
brak danych dla lotniska 11778 dnia 2019-05-01
brak danych dla lotniska 11778 dnia 2019-06-01
brak danych dla lotniska 11778 dnia 2019-07-01
brak danych dla lotniska 11778 dnia 2019-08-01
brak danych dla lotniska 11778 dnia 2019-09-01
brak danych dla lotniska 11778 dnia 2019-10-01
brak danych dla lotniska 11778 dnia 2019-11-01
brak danych dla lotniska 11778 dnia 2019-12-01
brak danych dla lotniska 11778 dnia 2020-01-01
brak danych dla lotniska 11778 dnia 2020-02-01
brak danych dla lotniska 11778 dnia 2020-03-01
dane dla lotniska 11778 pobrane
brak danych dla lotniska 12888 dnia 2019-01-01
brak danych dla lotniska 12888 dnia 2019-02-01
brak danych dla lotniska 12888 dnia 2019-03-01
brak danych dla lotniska 12888 dnia 2019-04-01
brak danych dla lotniska 12888 dnia 2019-05-01
brak danych dla lotniska 12888 dnia 2019-06-01
brak danych dla lotniska 12888 dnia 2019-07-01
brak danych dla lotniska 12888 dnia 2019-08-01
brak danych dla lotniska 128

brak danych dla lotniska 15624 dnia 2019-06-01
brak danych dla lotniska 15624 dnia 2019-07-01
brak danych dla lotniska 15624 dnia 2019-08-01
brak danych dla lotniska 15624 dnia 2019-09-01
brak danych dla lotniska 15624 dnia 2019-10-01
brak danych dla lotniska 15624 dnia 2019-11-01
brak danych dla lotniska 15624 dnia 2019-12-01
brak danych dla lotniska 15624 dnia 2020-01-01
brak danych dla lotniska 15624 dnia 2020-02-01
brak danych dla lotniska 15624 dnia 2020-03-01
dane dla lotniska 15624 pobrane
brak danych dla lotniska 10994 dnia 2019-01-01
brak danych dla lotniska 10994 dnia 2019-02-01
brak danych dla lotniska 10994 dnia 2019-03-01
brak danych dla lotniska 10994 dnia 2019-04-01
brak danych dla lotniska 10994 dnia 2019-05-01
brak danych dla lotniska 10994 dnia 2019-06-01
brak danych dla lotniska 10994 dnia 2019-07-01
brak danych dla lotniska 10994 dnia 2019-08-01
brak danych dla lotniska 10994 dnia 2019-09-01
brak danych dla lotniska 10994 dnia 2019-10-01
brak danych dla lotniska 109

brak danych dla lotniska 10558 dnia 2019-08-01
brak danych dla lotniska 10558 dnia 2019-09-01
brak danych dla lotniska 10558 dnia 2019-10-01
brak danych dla lotniska 10558 dnia 2019-11-01
brak danych dla lotniska 10558 dnia 2019-12-01
brak danych dla lotniska 10558 dnia 2020-01-01
brak danych dla lotniska 10558 dnia 2020-02-01
brak danych dla lotniska 10558 dnia 2020-03-01
dane dla lotniska 10558 pobrane
brak danych dla lotniska 10620 dnia 2019-01-01
brak danych dla lotniska 10620 dnia 2019-02-01
brak danych dla lotniska 10620 dnia 2019-03-01
brak danych dla lotniska 10620 dnia 2019-04-01
brak danych dla lotniska 10620 dnia 2019-05-01
brak danych dla lotniska 10620 dnia 2019-06-01
brak danych dla lotniska 10620 dnia 2019-07-01
brak danych dla lotniska 10620 dnia 2019-08-01
brak danych dla lotniska 10620 dnia 2019-09-01
brak danych dla lotniska 10620 dnia 2019-10-01
brak danych dla lotniska 10620 dnia 2019-11-01
brak danych dla lotniska 10620 dnia 2019-12-01
brak danych dla lotniska 106

brak danych dla lotniska 13795 dnia 2019-10-01
brak danych dla lotniska 13795 dnia 2019-11-01
brak danych dla lotniska 13795 dnia 2019-12-01
brak danych dla lotniska 13795 dnia 2020-01-01
brak danych dla lotniska 13795 dnia 2020-02-01
brak danych dla lotniska 13795 dnia 2020-03-01
dane dla lotniska 13795 pobrane
brak danych dla lotniska 10693 dnia 2019-01-01
brak danych dla lotniska 10693 dnia 2019-02-01
brak danych dla lotniska 10693 dnia 2019-03-01
brak danych dla lotniska 10693 dnia 2019-04-01
brak danych dla lotniska 10693 dnia 2019-05-01
brak danych dla lotniska 10693 dnia 2019-06-01
brak danych dla lotniska 10693 dnia 2019-07-01
brak danych dla lotniska 10693 dnia 2019-08-01
brak danych dla lotniska 10693 dnia 2019-09-01
brak danych dla lotniska 10693 dnia 2019-10-01
brak danych dla lotniska 10693 dnia 2019-11-01
brak danych dla lotniska 10693 dnia 2019-12-01
brak danych dla lotniska 10693 dnia 2020-01-01
brak danych dla lotniska 10693 dnia 2020-02-01
brak danych dla lotniska 106

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [1]:
flight_df.to_csv('flight.csv', index=False)

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [10]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
